In [ ]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction

df = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv")
df = df[df["Date"]>"2021-11-30"].copy()

env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    cur_date = prices.Date.iat[0]
    df = pd.concat([df, prices])
    df = df.sort_values(["SecuritiesCode", "Date"])
    df.loc[:, "Rate"] = ((df.Close.shift(-1)/df.Close.shift(-2) - 1)).shift(3).fillna(df.Target)
    df.Target.ffill(inplace=True)
    df.loc[:,"Rank"] = df.groupby("Date")["Target"].rank(ascending=False, method="first").astype(int)-1 
    pred = df[df['Date']==cur_date].set_index("SecuritiesCode")["Rank"]
    sample_prediction['Rank'] = sample_prediction["SecuritiesCode"].map(pred)
    env.predict(sample_prediction)